In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd
import numpy as np


In [2]:
# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Load your DataFrame
# Replace 'your_dataframe.csv' with the actual filename or method to load your data
df = pd.read_csv('Desktop/ML_Final_Project/train_dataframe.csv')
test_df = pd.read_csv('Desktop/ML_Final_Project/test_dataframe.csv')



In [4]:
# Assuming `df` is your training DataFrame and `test_df` is your testing DataFrame
X = df.drop('label', axis=1).values
y = df['label'].values

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Determine the number of classes and input shape
num_classes = len(set(y_encoded))
input_shape = X.shape[1]

# Split data into training and testing sets for training dataset
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

X_train_tensor = torch.Tensor(X_train).unsqueeze(1)  # shape: [batch_size, 1, sequence_length]
y_train_tensor = torch.Tensor(y_train).long()
X_val_tensor = torch.Tensor(X_val).unsqueeze(1)  # shape: [batch_size, 1, sequence_length]
y_val_tensor = torch.Tensor(y_val).long()

# Create dataset and dataloader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
X_train_tensor.shape

torch.Size([24555, 1, 23])

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out

class ResNetAudio(nn.Module):
    def __init__(self, block, layers, num_classes):
        super(ResNetAudio, self).__init__()
        self.in_channels = 16
        self.conv = nn.Conv1d(1, 16, kernel_size=7, stride=1, padding=3)
        self.bn = nn.BatchNorm1d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2)  # Additional layer
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, num_classes)  # Adjust the input size of the fully connected layer

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm1d(out_channels),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)  # Additional layer
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x




In [6]:
import torch
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)


In [7]:
num_classes = num_classes  # Adjust as per your dataset
hidden_size = 128  # LSTM hidden size
num_layers = 2  # Number of LSTM layers
model = ResNetAudio(ResidualBlock, [2, 2, 2, 2], num_classes=num_classes) 
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 300
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train_predictions = 0
    total_train_predictions = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Accumulate training loss and accuracy
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_train_predictions += (predicted == labels).sum().item()
        total_train_predictions += labels.size(0)

    train_accuracy = correct_train_predictions / total_train_predictions * 100

    # Validation accuracy
    if epoch % 20 == 0:
        model.eval()  # Set the model to evaluation mode
        correct_val_predictions = 0
        total_val_predictions = 0
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            _, predicted = torch.max(val_outputs, 1)
            correct_val_predictions += (predicted == y_val_tensor).sum().item()
            total_val_predictions += y_val_tensor.size(0)

        test_accuracy = correct_val_predictions / total_val_predictions * 100
        model.train()  # Set the model back to training mode

        print(f"Epoch {epoch + 1}, Loss: {running_loss / total_train_predictions}, Train Accuracy: {train_accuracy}%, Val Accuracy: {test_accuracy}%")


Epoch 1, Loss: 0.06479443612445325, Train Accuracy: 15.634290368560375%, Val Accuracy: 12.054080469131781%
Epoch 21, Loss: 0.024037818104860085, Train Accuracy: 62.59824882915903%, Val Accuracy: 52.712168105554646%
Epoch 41, Loss: 0.008926553282802544, Train Accuracy: 84.93585827733659%, Val Accuracy: 70.89102459683988%
Epoch 61, Loss: 0.00761564723560194, Train Accuracy: 88.67440439828955%, Val Accuracy: 72.2430363251344%
Epoch 81, Loss: 0.0037347754620765078, Train Accuracy: 94.0541641213602%, Val Accuracy: 72.03127545202803%
Epoch 101, Loss: 0.002053536510138997, Train Accuracy: 96.48951333740582%, Val Accuracy: 71.15165336374002%
Epoch 121, Loss: 0.0017154984504473344, Train Accuracy: 97.39767868051314%, Val Accuracy: 75.37058152793614%
Epoch 141, Loss: 0.0014419075189073646, Train Accuracy: 97.8415801262472%, Val Accuracy: 72.68284736927838%
Epoch 161, Loss: 0.0014916237758879245, Train Accuracy: 97.55650580329872%, Val Accuracy: 65.33637400228051%
Epoch 181, Loss: 0.0008496173004

In [8]:
from sklearn.metrics import classification_report
# Prepare the test dataset
X_test = test_df.drop('label', axis=1).values
y_test = test_df['label'].values
y_test_encoded = label_encoder.transform(y_test)  # Use the same encoder as before

# Convert test data to PyTorch tensors
X_test_tensor = torch.Tensor(X_test).unsqueeze(1)  # shape: [batch_size, 1, sequence_length]
y_test_tensor = torch.Tensor(y_test_encoded).long()

# Evaluate the model on test dataset
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)
    predicted = predicted.numpy()
    y_test_encoded = y_test_encoded.astype(int)

# Generate a classification report
report = classification_report(y_test_encoded, predicted, target_names=label_encoder.classes_)
print(report)

              precision    recall  f1-score   support

    eighties       0.83      1.00      0.91         5
     fifties       0.86      0.80      0.83       205
    fourties       0.74      0.80      0.77       236
   seventies       0.71      0.83      0.77        36
     sixties       0.90      0.86      0.88        88
       teens       0.71      0.73      0.72       117
    thirties       0.76      0.77      0.76       389
    twenties       0.81      0.79      0.80       466

    accuracy                           0.79      1542
   macro avg       0.79      0.82      0.81      1542
weighted avg       0.79      0.79      0.79      1542

